In [56]:
import pandas as pd
import random

historia:
loty startuja z ziemi - poczatek ukladu wspolrzednych.
im planeta w galaktyce dalej od ziemii tym:
- wiecej pracownikow
- drozszy lot
- mozliwe ze bedzie dostepny tylko w jedna strone

Im:
- statek ma wieksza capacity (wielkosc),
-
tym cena wyzsza

planety w jednej galaktyce maja zblizone wspolrzedne.


jak ktos pojechal tylko w jedna strone - to juz nie moze pojawic sie w transakcjach i bookingach.

niech salary pracownikow bedzie najwyzsze dla commander a najnizsze dla technitian i neich zalezy od doswiadczenia w firmie.



planets +  galaxy

In [61]:
import pandas as pd
import random

# Galaxies and their base coordinates (for clustering)
galaxies = [
    "Andros Spiral", "Helion Prime", "Novara Cluster", "Vortexis", "Celestara",
    "Omega Verge", "Halcyon Drift", "Nemera Arc", "Quasari Belt", "Eclipse Halo",
    "Astralis", "Kronis Ring", "Auroria", "Zerion Reach", "Polaris Nexus"
]

# Step 1: Generate galaxy table
galaxy_data = []
galaxy_coords = {}

for i, name in enumerate(galaxies, start=1):
    galaxy_data.append({
        "galaxy_id": i,
        "galaxy": name
    })
    # base coordinates for clustered planets
    galaxy_coords[i] = {
        "base_x": random.uniform(-1000, 1000),
        "base_y": random.uniform(-1000, 1000),
        "base_z": random.uniform(-1000, 1000)
    }

# Step 2: Generate planets clustered by galaxy
planet_names = [
    "Aetheria", "Zorath", "Nyxos", "Virella", "Targus", "Ecliptor", "Orionis", "Caelora",
    "Draconis", "Pyrrhos", "Talmera", "Lunaris", "Zephyra", "Solvana", "Obscura",
    "Cryon", "Valtor", "Serenos", "Arctyra", "Phobaris"
]

planet_data = []
for i, name in enumerate(planet_names):
    galaxy_id = random.randint(1, len(galaxies))
    base = galaxy_coords[galaxy_id]
    planet_data.append({
        "planet_id": i + 1,
        "planet": name,
        "x_coord": round(base["base_x"] + random.uniform(-100, 100), 2),
        "y_coord": round(base["base_y"] + random.uniform(-100, 100), 2),
        "z_coord": round(base["base_z"] + random.uniform(-100, 100), 2),
        "galaxy_id": galaxy_id
    })

# Convert to DataFrames
df_galaxies = pd.DataFrame(galaxy_data)
df_planets = pd.DataFrame(planet_data)

# Save to CSV
df_galaxies.to_csv("galaxy.csv", index=False)
df_planets.to_csv("planet.csv", index=False)

# # Show preview
# print("Preview of generated planet data:")
# print(df_planets.head())


to dodatkowa czesc Dominika

In [ ]:
# warsja rozszerzona o ruch orbitalny

import pandas as pd
import random
import numpy as np
from collections import defaultdict
import csv

random.seed(42)
np.random.seed(42)

# Nazwy galaktyk

with open('galaxies.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pomija nagłówek
    galaxies_example = [row[0] for row in reader]

# Losuj liczbę elementów między 12 a 15
num_to_select = random.randint(12, 15)

# Wybierz losowe, unikalne elementy z listy
galaxy_names = random.sample(galaxies_example, num_to_select)

# Nazwy systemów (bogowie, łacina, mitologia)

with open('systems.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pomija nagłówek
    system_names  = [row[0] for row in reader]


# Nazwy planet (mitologiczne, łacińskie)
with open('planets.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pomija nagłówek
    planet_names  = [row[0] for row in reader]

# Specjalne cechy
special_features_base = [
    "pink_sun", "comets_rain", "emerald_lakes", "diamond_rain", "1000*C",
    "huge_gravity", "sulfuric_acid_fog", "magnetic_storms", "glowing_mountains",
    "eternal_night", "frozen_oceans"
]

# === Inicjalizacja ===
galaxies = []
systems = []
planets = []

galaxy_coords = {}
system_coords = {}

system_id_counter = 1
planet_id_counter = 1

# Liczniki dla powtórzeń nazw
system_name_count = defaultdict(int)
planet_name_count = defaultdict(int)

# === GALAXY DATA ===
for i, name in enumerate(galaxy_names, start=1):
    galaxies.append({
        "galaxy_id": i,
        "galaxy": name
    })
    galaxy_coords[i] = {
        "base_x": random.uniform(-1000, 1000),
        "base_y": random.uniform(-1000, 1000),
        "base_z": random.uniform(-1000, 1000)
    }

# === PLANETARY SYSTEM DATA ===
for galaxy in galaxies:
    galaxy_id = galaxy["galaxy_id"]
    base = galaxy_coords[galaxy_id]

    num_systems = random.randint(2, 5)

    for _ in range(num_systems):
        raw_name = random.choice(system_names)
        system_name_count[raw_name] += 1
        name = raw_name if system_name_count[raw_name] == 1 else f"{raw_name}_{system_name_count[raw_name]}"

        x = round(base["base_x"] + random.uniform(-100, 100), 2)
        y = round(base["base_y"] + random.uniform(-100, 100), 2)
        z = round(base["base_z"] + random.uniform(-100, 100), 2)

        systems.append({
            "system_id": system_id_counter,
            "system": name,
            "central_star": f"{raw_name} Star",
            "x_coord": x,
            "y_coord": y,
            "z_coord": z,
            "galaxy_id": galaxy_id
        })

        system_coords[system_id_counter] = {"x": x, "y": y, "z": z}
        system_id_counter += 1

# === PLANET DATA ===
def generate_features():
    x = max(0, int(np.floor(np.random.normal(3, 2))))
    return random.sample(special_features_base, min(x, len(special_features_base)))

for system in systems:
    system_id = system["system_id"]
    base = system_coords[system_id]
    num_planets = random.randint(2, 7)

    for _ in range(num_planets):
        raw_name = random.choice(planet_names)
        planet_name_count[raw_name] += 1
        name = raw_name if planet_name_count[raw_name] == 1 else f"{raw_name}_{planet_name_count[raw_name]}"

        orbit_period_days = random.randint(50, 1000)
        features = generate_features()
        x0 = round(base["x"] + random.uniform(-10, 10), 2)
        y0 = round(base["y"] + random.uniform(-10, 10), 2)
        z0 = round(base["z"] + random.uniform(-10, 10), 2)

        planets.append({
            "planet_id": planet_id_counter,
            "planet": name,
            "system_id": system_id,
            "orbit_period_days": orbit_period_days,
            "special_features": ",".join(features),
            "x0_coord": x0,
            "y0_coord": y0,
            "z0_coord": z0
        })
        planet_id_counter += 1

# === DataFrames ===
df_galaxies = pd.DataFrame(galaxies)
df_systems = pd.DataFrame(systems)
df_planets = pd.DataFrame(planets)

# === Podgląd danych ===
print("Galaxies:")
print(df_galaxies.head(), "\n")

print("Systems:")
print(df_systems.head(), "\n")

print("Planets:")
print(df_planets.head(), "\n")

# # Eksport (opcjonalny)
# df_galaxies.to_csv("galaxies_from_diagram.csv", index=False)
# df_systems.to_csv("planetary_systems_from_diagram.csv", index=False)
# df_planets.to_csv("planets_from_diagram.csv", index=False)


Galaxies:
   galaxy_id           galaxy
0          1     Helion Prime
1          2   Solara Expanse
2          3  Velatrix Spiral
3          4    Polaris Nexus
4          5        Celestara 

Systems:
   system_id    system   central_star  x_coord  y_coord  z_coord  galaxy_id
0          1    Thalos    Thalos Star   -82.86  -964.38  -572.04          1
1          2   Nimbara   Nimbara Star  -223.63  -969.35  -528.69          1
2          3    Mithra    Mithra Star    17.54  -997.97  -609.87          2
3          4    Ignara    Ignara Star   108.62  -918.93  -590.93          2
4          5  Vulcanus  Vulcanus Star   245.58    -3.70  -596.03          3 

Planets:
   planet_id   planet  system_id  orbit_period_days  \
0          1  Juturna          1                 52   
1          2  Veridia          1                727   
2          3   Undina          1                112   
3          4   Dryope          2                921   
4          5   Themis          2                684   

 

In [27]:
df_planets

,planet_id,planet,system_id,orbit_period_days,special_features,x0_coord,y0_coord,z0_coord
0,1,Juturna,1,52,"sulfuric_acid_fog,1000*C,magnetic_storms",-87.16,-960.45,-567.43
1,2,Veridia,1,727,"magnetic_storms,emerald_lakes",-89.06,-970.03,-580.87
2,3,Undina,1,112,"huge_gravity,pink_sun,eternal_night,magnetic_s...",-82.80,-957.33,-578.89
3,4,Dryope,2,921,"emerald_lakes,comets_rain,eternal_night,diamon...",-214.80,-967.96,-527.11
4,5,Themis,2,684,"comets_rain,sulfuric_acid_fog",-220.48,-968.05,-532.36
...,...,...,...,...,...,...,...,...
167,168,Enyo,37,721,"frozen_oceans,sulfuric_acid_fog,emerald_lakes,...",-825.55,-468.63,-481.70
168,169,Tanaquil,37,145,"frozen_oceans,sulfuric_acid_fog",-822.91,-484.10,-491.78
169,170,Gorgona_2,38,834,magnetic_storms,-918.24,-461.27,-347.64
170,171,Drusilla_3,38,914,comets_rain,-911.93,-470.47,-335.97


In [28]:
df_systems

,system_id,system,central_star,x_coord,y_coord,z_coord,galaxy_id
0,1,Thalos,Thalos Star,-82.86,-964.38,-572.04,1
1,2,Nimbara,Nimbara Star,-223.63,-969.35,-528.69,1
2,3,Mithra,Mithra Star,17.54,-997.97,-609.87,2
3,4,Ignara,Ignara Star,108.62,-918.93,-590.93,2
4,5,Vulcanus,Vulcanus Star,245.58,-3.70,-596.03,3
5,6,Ignara_2,Ignara Star,213.01,172.51,-545.68,3
6,7,Axioma,Axioma Star,242.29,89.73,-482.18,3
7,8,Oracyn,Oracyn Star,228.34,17.81,-510.12,3
8,9,Quirinius,Quirinius Star,258.10,598.74,-1043.14,4
9,10,Lunaris,Lunaris Star,180.44,537.04,-1077.58,4


In [29]:
df_galaxies

,galaxy_id,galaxy
0,1,Helion Prime
1,2,Solara Expanse
2,3,Velatrix Spiral
3,4,Polaris Nexus
4,5,Celestara
5,6,Virelia Cluster
6,7,Vortexis
7,8,Aetherion Swirl
8,9,Zenithis Ring
9,10,Crystara Bloom


trip types

In [62]:
import math
import pandas as pd

# Zakładam, że df_planets już istnieje (jeśli nie, trzeba go wcześniej załadować z CSV)

# Step 1: Generate a trip type for each planet
trip_types = []

for i, row in df_planets.iterrows():
    distance_from_origin = math.sqrt(row["x_coord"]**2 + row["y_coord"]**2 + row["z_coord"]**2)
    is_round_trip = distance_from_origin < 900  # one-way if far from (0,0,0)
    
    trip_types.append({
        "triptype_id": i + 1,
        "name": f"Mission to {row['planet']}",
        #"coordinates": f"X:{row['x_coord']}, Y:{row['y_coord']}, Z:{row['z_coord']}",
        "description": f"Explore the unique surface and atmosphere of planet {row['planet']}.",
        "is_round_trip": is_round_trip,
        "planet_id": row["planet_id"]
    })

# Convert to DataFrame and save
df_trip_types = pd.DataFrame(trip_types)

# Zmień ścieżkę na swoją lokalną, np. na Pulpit
#df_trip_types.to_csv("trip_types_linked_to_planets.csv", index=False)

# Show previe


In [63]:
#df_trip_types
df_trip_types.to_csv("triptype.csv", index=False)

rocket

In [64]:
import pandas as pd

# Dane o rakietach
rocket_data = [
    {"rocket_id": 1, "name": "Stellar Vortex", "capacity": 6, "max_range": 5000},
    {"rocket_id": 2, "name": "Nebula Strider", "capacity": 10, "max_range": 15000},
    {"rocket_id": 3, "name": "Quantum Lance", "capacity": 4, "max_range": 3000},
    {"rocket_id": 4, "name": "Solaris Ark", "capacity": 8, "max_range": 10000},
]

# DataFrame
df_rockets = pd.DataFrame(rocket_data)

# Zapis do pliku CSV
df_rockets.to_csv("rocket.csv", index=False)


employees

In [65]:
import random
import pandas as pd
from datetime import datetime, timedelta
from faker import Faker

fake = Faker()

positions = ["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"]
position_weights = [1, 1, 1, 3, 3]

salary_ranges = {
    "Technician": (4000, 6000),
    "Mission Specialist": (8000, 11000),
    "Commander": (9000, 12000),
    "Pilot": (6000, 8500),
    "Navigator": (5500, 8000)
}

# Parametry czasu
start_date = datetime(2023, 1, 1)
end_random_hire = datetime(2025, 6, 1)
today = datetime.today()

employees_data = []
num_employees = 30
pct_early_hires = 0.5
early_hires = int(num_employees * pct_early_hires)

for i in range(num_employees):
    first_name = fake.first_name()
    last_name = fake.last_name()

    position = random.choices(positions, weights=position_weights, k=1)[0]
    low, high = salary_ranges[position]

    # Hire date
    if i < early_hires:
        hire_date = start_date
    else:
        rand_days = random.randint(0, (end_random_hire - start_date).days)
        hire_date = start_date + timedelta(days=rand_days)

    # Staż w latach (pełne lata)
    years_worked = max(0, (today - hire_date).days // 365)

    # Wynagrodzenie bazowe + podwyżki
    base_salary = random.randrange(low, high + 1, 100)
    adjusted_salary = round(base_salary * (1.07 ** years_worked), -2)  # zaokrąglenie do setek

    employees_data.append({
        "employee_id": i + 1,
        "first_name": first_name,
        "last_name": last_name,
        "position": position,
        "hire_date": hire_date.strftime("%Y-%m-%d"),
        "salary": adjusted_salary
    })

# DataFrame
df_employees = pd.DataFrame(employees_data)



In [66]:
df_employees.to_csv("employee.csv", index=False)
# df_employees

trip + tripemployees

In [67]:
def is_employee_available(emp_id, new_start, new_end):
    for trip in trip_employees:
        if trip["employee_id"] == emp_id:
            # znajdź zakres dat tej misji
            trip_id = trip["trip_id"]
            trip_info = next((t for t in trips if t["trip_id"] == trip_id), None)
            if trip_info:
                existing_start = datetime.strptime(trip_info["departure_date"], "%Y-%m-%d")
                existing_end = datetime.strptime(trip_info["return_date"], "%Y-%m-%d")
                # sprawdź konflikt
                if not (new_end < existing_start or new_start > existing_end):
                    return False
    return True


In [68]:
# Update employee and trip generation to ensure each trip has a Pilot and Navigator
from datetime import datetime
from datetime import timedelta

rockets=['Stellar Vortex','Nebula Strider','Quantum Lance','Solaris Ark']
employees=["Technician", "Mission Specialist", "Commander","Pilot", "Navigator"]
start_date=datetime(2023,1,1)
trips = []
trip_employees = []

for i, row in df_trip_types.iterrows():
    num_variants = random.randint(1, 3)  # 1-3 trips per trip type
    for _ in range(num_variants):
        trip_id = len(trips) + 1
        departure = start_date + timedelta(days=random.randint(0,600))
        duration = random.randint(5, 30)
        return_date = departure + timedelta(days=duration)
        rocket_id = random.randint(1, len(rockets))

        trips.append({
            "trip_id": trip_id,
            "triptype_id": row["triptype_id"],
            "rocket_id": rocket_id,
            "departure_date": departure.strftime("%Y-%m-%d"),
            "return_date": return_date.strftime("%Y-%m-%d"),
            "status": random.choice(["Scheduled", "Completed", "Delayed"])
        })

        is_round_trip = row["is_round_trip"]
        min_roles = ["Pilot", "Navigator"]
        other_roles = ["Technician", "Mission Specialist", "Commander"]
        n_optional = random.randint(1, 2) if is_round_trip else 0  # more if round trip

        assigned = []
        # Add required roles
        # Dodaj obowiązkowego Pilota i Nawigatora
        for role in min_roles:
            available = [e for e in employees_data if e["position"] == role and e["employee_id"] not in assigned and is_employee_available(e["employee_id"], departure, return_date)]
        if not available:
            raise ValueError(f"No available employee for role: {role} (without conflict)")
    
        emp = random.choice(available)
        assigned.append(emp["employee_id"])
        trip_employees.append({
        "trip_id": trip_id,
        "employee_id": emp["employee_id"],
        "role_in_trip": emp["position"]
    })


        # Add optional roles
        for _ in range(n_optional):
            available = [e for e in employees_data if e["position"] in other_roles and e["employee_id"] not in assigned]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                trip_employees.append({
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })

# Update DataFrames and save
df_trips = pd.DataFrame(trips)
df_trip_employees = pd.DataFrame(trip_employees)

df_trips.to_csv("trip.csv", index=False)
df_trip_employees.to_csv("tripemployees.csv", index=False)


# Preview assigned trip employees
#import ace_tools as tools; tools.display_dataframe_to_user(name="Trip Employees with Required Roles", dataframe=df_trip_employees)


In [26]:
df_trip_employees

,trip_id,employee_id,role_in_trip
0,1,3,Navigator
1,2,5,Navigator
2,3,3,Navigator
3,4,18,Navigator
4,5,18,Navigator
...,...,...,...
64,40,13,Mission Specialist
65,41,19,Navigator
66,41,27,Commander
67,42,19,Navigator


wersja 2

In [69]:
from datetime import datetime, timedelta
import random
import math
import pandas as pd

rockets = ['Stellar Vortex', 'Nebula Strider', 'Quantum Lance', 'Solaris Ark']
employees = ["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"]
start_date = datetime(2023, 1, 1)
trips = []
trip_employees = []
planet_coords = df_planets.set_index("planet_id")[["x_coord", "y_coord", "z_coord"]].to_dict(orient="index")

# Rakiety z parametrami (zasięg itd.)
rocket_data = {
    1: {"name": "Stellar Vortex", "max_range": 5000},
    2: {"name": "Nebula Strider", "max_range": 15000},
    3: {"name": "Quantum Lance", "max_range": 3000},
    4: {"name": "Solaris Ark", "max_range": 10000},
}

for i, row in df_trip_types.iterrows():
    num_variants = random.randint(1, 3)
    for _ in range(num_variants):
        departure = start_date + timedelta(days=random.randint(0, 600))
        duration = random.randint(5, 30)
        return_date = departure + timedelta(days=duration)
        planet_id = row["planet_id"]

        # Oblicz dystans do planety
        coords = planet_coords[planet_id]
        distance = math.sqrt(coords["x_coord"]**2 + coords["y_coord"]**2 + coords["z_coord"]**2)

        is_round_trip = row["is_round_trip"]
        required_range = distance * (2 if is_round_trip else 1)

        # Szukaj rakiety o wystarczającym zasięgu
        suitable_rockets = [rid for rid, data in rocket_data.items() if data["max_range"] >= required_range]
        if not suitable_rockets:
            continue  # pomiń podróż jeśli żadna rakieta nie pasuje

        rocket_id = random.choice(suitable_rockets)
        rocket_name = rocket_data[rocket_id]["name"]

        trip_id = len(trips) + 1

        trips.append({
            "trip_id": trip_id,
            "triptype_id": row["triptype_id"],
            "rocket_id": rocket_id,
            "departure_date": departure.strftime("%Y-%m-%d"),
            "return_date": return_date.strftime("%Y-%m-%d"),
            "status": random.choice(["Scheduled", "Completed", "Delayed"])
        })

        # Załoga
        min_roles = ["Pilot", "Navigator"]
        other_roles = ["Technician", "Mission Specialist", "Commander"]
        n_optional = random.randint(1, 2) if is_round_trip else 0

        assigned = []

        for role in min_roles:
            available = [
                e for e in employees_data
                if e["position"] == role
                and e["employee_id"] not in assigned
                and is_employee_available(e["employee_id"], departure, return_date)
            ]
            if not available:
                raise ValueError(f"No available employee for role: {role} (without conflict)")

            emp = random.choice(available)
            assigned.append(emp["employee_id"])
            trip_employees.append({
                "trip_id": trip_id,
                "employee_id": emp["employee_id"],
                "role_in_trip": emp["position"]
            })

        for _ in range(n_optional):
            available = [
                e for e in employees_data
                if e["position"] in other_roles and e["employee_id"] not in assigned
                and is_employee_available(e["employee_id"], departure, return_date)
            ]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                trip_employees.append({
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })

# DataFrames
df_trips = pd.DataFrame(trips)
df_trip_employees = pd.DataFrame(trip_employees)

# Zapisz do plików
df_trips.to_csv("trip.csv", index=False)
df_trip_employees.to_csv("tripemployees.csv", index=False)


najbardziej sie spoznia - quantum lance

In [ ]:
# from datetime import datetime, timedelta
# import random
# import math
# import pandas as pd

# # Dane bazowe
# rockets = ['Stellar Vortex', 'Nebula Strider', 'Quantum Lance', 'Solaris Ark']
# employees_data = df_employees[["employee_id", "position"]].to_dict(orient="records")

# start_date = datetime(2023, 1, 1)
# cutoff_date = datetime(2025, 6, 1)
# delayed_rocket = "Quantum Lance"

# # Rakiety z zasięgiem
# rocket_data = [
#     {"rocket_id": 1, "name": "Stellar Vortex", "capacity": 6, "max_range": 5000},
#     {"rocket_id": 2, "name": "Nebula Strider", "capacity": 10, "max_range": 15000},
#     {"rocket_id": 3, "name": "Quantum Lance", "capacity": 4, "max_range": 3000},
#     {"rocket_id": 4, "name": "Solaris Ark", "capacity": 8, "max_range": 10000},
# ]

# df_rockets = pd.DataFrame(rocket_data)

# # Przygotowanie
# planet_coords = df_planets.set_index("planet_id")[["x_coord", "y_coord", "z_coord"]].to_dict(orient="index")

# #planet_coords = df_planets.set_index("planet_id")[["x_coords", "y_coords", "z_coords"]].to_dict(orient="index")
# trips = []
# trip_employees = []

# def is_employee_available(emp_id, start, end):
#     for t in trip_employees:
#         if t["employee_id"] == emp_id:
#             trip_id = t["trip_id"]
#             trip = next(tr for tr in trips if tr["trip_id"] == trip_id)
#             d_start = datetime.strptime(trip["departure_date"], "%Y-%m-%d")
#             d_end = datetime.strptime(trip["return_date"], "%Y-%m-%d")
#             if not (end < d_start or start > d_end):  # overlap
#                 return False
#     return True

# for i, row in df_trip_types.iterrows():
#     num_variants = random.randint(1, 3)
#     for _ in range(num_variants):
#         departure = start_date + timedelta(days=random.randint(0, 1500))
#         duration = random.randint(5, 30)
#         return_date = departure + timedelta(days=duration)

#         # wybieramy losową rakietę i sprawdzamy czy doleci
#         tries = 0
#         while tries < 10:
#             rocket_id = random.randint(1, len(rockets))
#             rocket_info = rocket_data[rocket_id - 1]
#             rocket_name = rocket_info["name"]
#             rocket_range = rocket_info["max_range"]

#             # oblicz dystans do planety
#             planet_id = row["planet_id"]
#             coords = planet_coords[planet_id]
#             distance = math.sqrt(coords["x_coord"]**2 + coords["y_coord"]**2 + coords["z_coord"]**2)

#             if distance * 2 <= rocket_range:
#                 break  # rakieta da radę
#             tries += 1
#         else:
#             continue  # żadna rakieta nie pasuje, pomijamy tę podróż

#         trip_id = len(trips) + 1

#         if return_date < cutoff_date:
#             status = "Completed"
#         elif rocket_name == delayed_rocket:
#             status = "Delayed"
#         else:
#             status = random.choice(["Scheduled", "Delayed"])

#         trips.append({
#             "trip_id": trip_id,
#             "triptype_id": row["triptype_id"],
#             "rocket_id": rocket_id,
#             "departure_date": departure.strftime("%Y-%m-%d"),
#             "return_date": return_date.strftime("%Y-%m-%d"),
#             "status": status
#         })

#         is_round_trip = row["is_round_trip"]
#         min_roles = ["Pilot", "Navigator"]
#         other_roles = ["Technician", "Mission Specialist", "Commander"]
#         n_optional = random.randint(1, 2) if is_round_trip else 0

#         assigned = []
#         for role in min_roles:
#             available = [
#                 e for e in employees_data
#                 if e["position"] == role
#                 and e["employee_id"] not in assigned
#                 and is_employee_available(e["employee_id"], departure, return_date)
#             ]

#             if not available:
#                 raise ValueError(f"No available employee for role: {role} (without conflict)")

#             emp = random.choice(available)
#             assigned.append(emp["employee_id"])
#             trip_employees.append({
#                 "trip_id": trip_id,
#                 "employee_id": emp["employee_id"],
#                 "role_in_trip": emp["position"]
#             })

#         for _ in range(n_optional):
#             available = [
#                 e for e in employees_data
#                 if e["position"] in other_roles and e["employee_id"] not in assigned
#                 and is_employee_available(e["employee_id"], departure, return_date)
#             ]
#             if available:
#                 emp = random.choice(available)
#                 assigned.append(emp["employee_id"])
#                 trip_employees.append({
#                     "trip_id": trip_id,
#                     "employee_id": emp["employee_id"],
#                     "role_in_trip": emp["position"]
#                 })

# # DataFrames
# df_trips = pd.DataFrame(trips)
# df_trip_employees = pd.DataFrame(trip_employees)
# df_employees = pd.DataFrame(employees_data)

# # Save to CSV
# df_trips.to_csv("trips_generated.csv", index=False)
# df_trip_employees.to_csv("trip_employees.csv", index=False)
# df_employees.to_csv("employees_generated.csv", index=False)
# df_rockets.to_csv("rockets_generated.csv", index=False)


In [ ]:
# df_trips.to_csv("trips_generated.csv", index=False)
# df_trip_employees.to_csv("trip_employees.csv", index=False)
# df_employees.to_csv("employees_generated.csv", index=False)
# df_rockets.to_csv("rockets_generated.csv", index=False)

client + adress + emergency_contact

In [70]:


from faker import Faker
import pandas as pd

fake = Faker()

# 1. Klienci
clients = []
addresses = []
emergency_contacts = []

for i in range(50):
    client_id = i + 1

    # Klient
    first_name = fake.first_name()
    last_name = fake.last_name()

    # Adres
    address_id = client_id  # zakładamy 1:1
    address_phone = fake.numerify("#########")

    email_username = f"{first_name}.{last_name}".replace(" ", "")
    domain = fake.free_email_domain()
    address_email = f"{email_username}@{domain}"

    # Kontakt awaryjny
    emergency_contact_id = client_id  # zakładamy 1:1
    emergency_phone = fake.numerify("#########")

    email_username = f"{fake.first_name()}.{fake.last_name()}".replace(" ", "")
    domain = fake.free_email_domain()
    emergency_email = f"{email_username}@{domain}"

    clients.append({
        "client_id": client_id,
        "first_name": first_name,
        "last_name": last_name,
        "address_id": address_id,
        "emergency_contact_id": emergency_contact_id
    })

    addresses.append({
        "address_id": address_id,
        #"client_id": client_id,
        "phone": address_phone,
        "email": address_email
    })

    emergency_contacts.append({
        "emergency_contact_id": emergency_contact_id,
        #"client_id": client_id,
        "phone": emergency_phone,
        "email": emergency_email
    })

# 2. DataFrame'y
df_clients = pd.DataFrame(clients)
df_addresses = pd.DataFrame(addresses)
df_emergency_contacts = pd.DataFrame(emergency_contacts)

# 3. Zapis do CSV (opcjonalnie)
df_clients.to_csv("client.csv", index=False)
df_addresses.to_csv("address.csv", index=False)
df_emergency_contacts.to_csv("emergency_contacts.csv", index=False)


 booking

In [76]:
# Krok 0: przygotowanie mapy: trip_id -> is_round_trip

rockets_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()

triptype_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()
trip_is_round_trip = df_trips.set_index("trip_id")["triptype_id"].map(triptype_flags).to_dict()

# Zestaw klientów, którzy już pojechali w jedną stronę
clients_lost_in_space = set()

bookings = []
booking_id = 1
trip_seat_counts = {trip_id: 0 for trip_id in df_trips["trip_id"].tolist()}

for client in clients:
    if client["client_id"] in clients_lost_in_space:
        continue  # ten klient już odleciał w jedną stronę

    n_bookings = random.randint(1, 3)
    booked_trip_ids = random.sample(df_trips["trip_id"].tolist(), n_bookings)

    for trip_id in booked_trip_ids:
        if client["client_id"] in clients_lost_in_space:
            break  # klient już odleciał podczas tej iteracji

        rocket_id = df_trips.loc[df_trips["trip_id"] == trip_id, "rocket_id"].values[0]
        max_seats = rockets_capacity[rocket_id]
        seats_taken = trip_seat_counts[trip_id]
        seats_left = max_seats - seats_taken

        if seats_left <= 0:
            continue

        num_seats = random.randint(1, min(4, seats_left))
        for seat in range(1, num_seats + 1):
            bookings.append({
        "booking_id": booking_id,
        "client_id": client["client_id"],
        "trip_id": trip_id,
        "seat_number": seats_taken + seat
    })
            booking_id += 1  # teraz inkrementujesz osobno dla każdego miejsca
        trip_seat_counts[trip_id] += num_seats


        # Jeśli to podróż w jedną stronę — klient znika!
        if not trip_is_round_trip[trip_id]:
            clients_lost_in_space.add(client["client_id"])




df_bookings = pd.DataFrame(bookings)
df_bookings.to_csv("booking.csv", index=False)

transaction

In [77]:

#planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()
triptypes_planet = df_trip_types.set_index("triptype_id")["planet_id"].to_dict()
rockets_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()

# Count number of bookings per trip
#booking_counts = df_bookings.groupby("trip_id").size().to_dict()
# Liczba miejsc zarezerwowanych przez danego klienta na konkretny lot
booking_seat_counts = (
    df_bookings.groupby(["client_id", "trip_id"]).size().to_dict()
)

# Przygotuj słownik: planet_id -> (x, y, z)
planet_coords = df_planets.set_index("planet_id")[["x_coord", "y_coord", "z_coord"]].to_dict(orient="index")

transactions = []
for booking in bookings:
    trip_id = booking["trip_id"]
    trip_row = df_trips[df_trips["trip_id"] == trip_id].iloc[0]
    triptype_id = trip_row["triptype_id"]
    rocket_id = trip_row["rocket_id"]

    planet_id = triptypes_planet[triptype_id]
    coords = planet_coords[planet_id]  # np. {'x': 1.0, 'y': 2.0, 'z': 3.0}
    distance = math.sqrt(coords["x_coord"]**2 + coords["y_coord"]**2 + coords["z_coord"]**2)

    employee_count = df_trip_employees[df_trip_employees["trip_id"] == trip_id].shape[0]
# USTALANIE CENY
    base_price = 50000
    key = (booking["client_id"], trip_id)
    base_price += booking_seat_counts.get(key, 1) * 5000

    base_price += round(distance*5,-3)
    base_price += rockets_capacity[rocket_id] * 1000
    base_price += employee_count * 5000
    if round_trip_flags[triptype_id]:
        base_price *= 1.5

    key = (booking["client_id"], trip_id)
    if key not in {(t["client_id"], t["trip_id"]) for t in transactions}:
        num_seats = booking_seat_counts.get(key, 1)
        amount = round(base_price * num_seats, 2)

        transactions.append({
            "transaction_id": len(transactions) + 1,
            "client_id": booking["client_id"],
            "trip_id": trip_id,
            "num_seats": num_seats,
            "amount": amount,
            "payment_date": fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
            "method": random.choice(["card", "transfer", "crypto"])
        })

df_transactions = pd.DataFrame(transactions)


# Save all new CSVs
# df_clients.to_csv("clients_generated.csv", index=False)
# df_bookings.to_csv("bookings_generated.csv", index=False)
# df_transactions.to_csv("transactions_generated.csv", index=False)


In [52]:
df_transactions

,transaction_id,client_id,trip_id,num_seats,amount,payment_date,method
0,1,1,10,4,372000.0,2024-06-27,card
1,2,2,31,4,356000.0,2025-03-30,transfer
2,3,3,6,3,258000.0,2025-03-21,crypto
3,4,4,20,4,388000.0,2024-07-23,card
4,5,5,3,2,158000.0,2025-03-04,crypto
5,6,6,26,2,170000.0,2024-07-17,transfer
6,7,7,5,3,252000.0,2024-12-28,transfer
7,8,8,17,4,552000.0,2025-03-05,crypto
8,9,8,39,4,364000.0,2024-12-09,transfer
9,10,9,40,4,364000.0,2025-02-02,crypto


In [78]:

df_transactions.to_csv("transaction.csv", index=False)

cost

In [79]:

# Count employees per trip
employee_counts = df_trip_employees.groupby("trip_id").size().to_dict()
# Sum of transactions per trip
income_per_trip = df_transactions.groupby("trip_id")["amount"].sum().to_dict()
# Triptype info
planet_coords = df_planets.set_index("planet_id")[["x_coord", "y_coord", "z_coord"]].to_dict(orient="index")

#planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()

costs = []
for trip in df_trips.itertuples():
    trip_id = trip.trip_id
    triptype_id = trip.triptype_id
    planet_id = triptypes_planet[triptype_id]
    coords = planet_coords[planet_id]  # np. {'x': 1.0, 'y': 2.0, 'z': 3.0}
    distance = math.sqrt(coords["x_coord"]**2 + coords["y_coord"]**2 + coords["z_coord"]**2)

    #coords = planet_coords[triptype_id]
    #coord_parts = [float(val.split(":")[1]) for val in coords.split(", ")]
    #distance = math.sqrt(sum(c ** 2 for c in coord_parts))
    is_round = round_trip_flags[triptype_id]
    n_employees = employee_counts.get(trip_id, 1)
    max_total_cost = income_per_trip.get(trip_id, 10000)

    # Base cost estimate
    base_cost = 40000 + (n_employees * 10000) + round(distance*10, -3)
    if is_round:
        base_cost *= 1.5

    # Generate 2-4 individual costs that sum up to less than income
    total_cost = base_cost
    #min(base_cost, max_total_cost * 0.9)
    n_costs = random.randint(2, 4)
    cost_shares = [random.uniform(0.15, 0.35) for _ in range(n_costs)]
    total_share = sum(cost_shares)
    normalized_shares = [s / total_share for s in cost_shares]

    for i in range(n_costs):
        costs.append({
            "cost_id": len(costs) + 1,
            "trip_id": trip_id,
            "cost_type": random.choice(["Fuel", "Crew", "Maintenance", "Logistics"]),
            "amount": round(total_cost * normalized_shares[i], 2)
        })

df_costs = pd.DataFrame(costs)
# df_costs.to_csv("/mnt/data/costs_generated.csv", index=False)

# # Preview costs
# import ace_tools as tools; tools.display_dataframe_to_user(name="Generated Costs", dataframe=df_costs)


In [60]:
df_costs

,cost_id,trip_id,cost_type,amount
0,1,1,Crew,16384.42
1,2,1,Fuel,15424.15
2,3,1,Fuel,22819.07
3,4,1,Logistics,20372.37
4,5,2,Fuel,16970.94
...,...,...,...,...
126,127,39,Logistics,42013.33
127,128,39,Fuel,27986.67
128,129,40,Crew,18830.02
129,130,40,Logistics,25429.70


In [80]:
df_costs.to_csv("cost.csv", index=False)